In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.11:1.4.0,com.datastax.spark:spark-cassandra-connector_2.11:1.6.0-M2 pyspark-shell'
import time

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
#sc.stop()
conf = SparkConf() \
    .setAppName("Cassandra connection") \
    .setMaster("local[4]") \
    .set("spark.cassandra.connection.host", "localhost")

sc = SparkContext(conf=conf) 


In [4]:
sqlContext=SQLContext(sc)

In [5]:
from pyspark.sql.types import *
fields = [StructField("card_pan_id", StringType(), False),\
          StructField("term_miduid", StringType(), False),\
          StructField("term_mid", StringType(), False),\
          StructField("term_uid", StringType(), False),\
          StructField("term_mcc", StringType(), False),\
          StructField("term_country", StringType(), False),\
          StructField("tx_amount", StringType(), False),\
          StructField("tx_datetime", StringType(), False),\
          StructField("tx_accepted", StringType(), False),\
          StructField("tx_ecom", StringType(), False),\
          StructField("tx_3d_secure", StringType(), False),\
          StructField("tx_emv", StringType(), False),\
          StructField("age", StringType(), False),\
          StructField("language", StringType(), False),\
          StructField("gender", StringType(), False),\
          StructField("broker", StringType(), False),\
          StructField("card_brand", StringType(), False),\
          StructField("tx_fraud", StringType(), False),\
        ]
schema = StructType(fields)

In [6]:
csvfile = sqlContext.read.format('com.databricks.spark.csv',).schema(schema).load("20160322stream_subset100.csv")
csvfile.registerTempTable("txData");
csvfile.printSchema()

root
 |-- card_pan_id: string (nullable = false)
 |-- term_miduid: string (nullable = false)
 |-- term_mid: string (nullable = false)
 |-- term_uid: string (nullable = false)
 |-- term_mcc: string (nullable = false)
 |-- term_country: string (nullable = false)
 |-- tx_amount: string (nullable = false)
 |-- tx_datetime: string (nullable = false)
 |-- tx_accepted: string (nullable = false)
 |-- tx_ecom: string (nullable = false)
 |-- tx_3d_secure: string (nullable = false)
 |-- tx_emv: string (nullable = false)
 |-- age: string (nullable = false)
 |-- language: string (nullable = false)
 |-- gender: string (nullable = false)
 |-- broker: string (nullable = false)
 |-- card_brand: string (nullable = false)
 |-- tx_fraud: string (nullable = false)



In [7]:
start = time.time()
df=sqlContext.sql("select * from txData limit 10000")
df=df.repartition(20)
df.cache()
end = time.time()
print(end - start)


0.851726055145


In [9]:
start = time.time()
df.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('overwrite')\
    .options(table="rawdata", keyspace="tx")\
    .save()
end = time.time()
print(end - start)


6.75174283981


In [10]:
dd=sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="rawdata", keyspace="tx")\
    .load()
dd.registerTempTable("txData2");

In [11]:
dd.count()

10000

In [12]:
dd2=sqlContext.sql("select * from txData2 where card_brand='VIS'")
dd2.count()


4670